In [ ]:
import cv2
import numpy as np
import os

def compute_temporal_average(video_path, output_path, batch_size=100, temp_dir="temp_batches"):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    
    batch_index = 0
    count_per_batch = []
    
    while True:
        batch_frames = []
        for _ in range(batch_size):
            ret, frame = cap.read()
            if not ret:
                break
            batch_frames.append(frame.astype(np.float32))
        
        if not batch_frames:
            break
        
        batch_avg = np.mean(batch_frames, axis=0)
        batch_file = os.path.join(temp_dir, f"batch_{batch_index}.npy")
        np.save(batch_file, batch_avg)
        count_per_batch.append(len(batch_frames))
        batch_index += 1
    
    cap.release()
    
    if batch_index == 0:
        print("Error: No frames processed.")
        return
    
    final_avg = np.zeros((height, width, 3), dtype=np.float32)
    total_count = sum(count_per_batch)
    
    for i in range(batch_index):
        batch_avg = np.load(os.path.join(temp_dir, f"batch_{i}.npy"))
        final_avg += batch_avg * count_per_batch[i]
    
    final_avg /= total_count
    final_avg = np.clip(final_avg, 0, 255).astype(np.uint8)
    
    cv2.imwrite(output_path, final_avg)
    print(f"Temporal average saved as {output_path}")
    
    # Cleanup temporary files
    for i in range(batch_index):
        os.remove(os.path.join(temp_dir, f"batch_{i}.npy"))
    os.rmdir(temp_dir)

if __name__ == "__main__":
    video_path = "video.mp4"
    output_path = "output_image.jpg"
    compute_temporal_average(video_path, output_path)
